In [1]:
import csv
import requests
import os
import gzip
import shutil
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
countries= ["Spain","Italy", "Portugal"]

In [3]:
url="http://insideairbnb.com/get-the-data.html"
html=urlopen(url)
soup=BeautifulSoup(html, "html.parser")    

## Check city names to download

In [4]:
for item in soup.find_all('h2'):
    string=item.get_text()
    split_string=string.split(", ")
    country=split_string[-1]
    if country in countries:
        print(string)

Barcelona, Catalonia, Spain
Bergamo, Lombardia, Italy
Bologna, Emilia-Romagna, Italy
Euskadi, Euskadi, Spain
Florence, Toscana, Italy
Girona, Catalonia, Spain
Lisbon, Lisbon, Portugal
Madrid, Comunidad de Madrid, Spain
Malaga, Andalucía, Spain
Mallorca, Islas Baleares, Spain
Menorca, Islas Baleares, Spain
Milan, Lombardy, Italy
Naples, Campania, Italy
Porto, Norte, Portugal
Puglia, Puglia, Italy
Rome, Lazio, Italy
Sevilla, Andalucía, Spain
Sicily, Sicilia, Italy
Trentino, Trentino-Alto Adige/Südtirol, Italy
Valencia, Valencia, Spain
Venice, Veneto, Italy


## Download .csv.gz files to airbnb_gz

In [5]:
os.mkdir('airbnb_gz')

In [6]:
for item in soup.find_all('h2'):
    string=item.get_text()
    if string=='Trentino, Trentino-Alto Adige/Südtirol, Italy':
        string='Trentino, Trentino, Italy'
    if string=='Euskadi, Euskadi, Spain':
        string='Basque Country, Basque Country, Spain'
    if string=='Milan, Lombardy, Italy':
        string='Milan, Lombardia, Italy'
    split_string=string.split(', ')
    country=split_string[-1]
    if country in countries:
        table=item.nextSibling.nextSibling.nextSibling.nextSibling
        #print(table.get('class', []))
        #print(type(table))
        listing_url=table.contents[3].contents[1].contents[5].contents[0]['href']
        filename=f'airbnb_gz\{string}.csv.gz'

        with open(filename, "wb") as f:
            r = requests.get(listing_url)
            f.write(r.content)

## Unpack .gz to .csv in airbnb_csv

In [7]:
os.mkdir('airbnb_csv')

In [8]:
for filename in os.listdir('airbnb_gz'):
    with gzip.open(f'airbnb_gz\{filename}', 'rb') as f_in:
        new_filename=f'airbnb_csv\{filename[0:-3]}'
        with open(new_filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)